In [16]:
# Impor library
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import warnings

# untuk output yang lebih rapi
warnings.filterwarnings("ignore")

**1️. Tampilkan data yang telah diunduh, kemudian jelaskan semua kolom yang terdapat pada dataset tersebut.**

In [17]:
# Membaca data
feature_data = pd.read_csv("/content/features.csv")
label_data = pd.read_csv("/content/labels.csv")

# Menampilkan lima baris teratas dari data features
print("--- Tampilan Awal Data Features ---")
display(feature_data.head())

# Menampilkan lima baris teratas dari data labels
print("\n--- Tampilan Awal Data Target ---")
display(label_data.head())

--- Tampilan Awal Data Features ---


,Unnamed: 0,Student Id,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,0,6634,single,1.0,3.0,9773.0,day,1.0,NaN,Portuguese,...,0.0,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02
1,1,6459,single,1.0,1.0,NaN,day,1.0,136.0,Portuguese,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79
2,2,2238,single,42.0,1.0,9500.0,day,1.0,120.0,Portuguese,...,0.0,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70
3,3,2479,single,1.0,1.0,NaN,day,1.0,141.0,Portuguese,...,0.0,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02
4,4,4260,single,44.0,1.0,NaN,day,39.0,120.0,Spanish,...,0.0,1.0,6.0,NaN,3.0,12.000000,0.0,12.4,0.5,1.79



--- Tampilan Awal Data Target ---


,Target
0,Graduate
1,Graduate
2,Graduate
3,Graduate
4,Dropout


1.Student Id: ID unik untuk setiap mahasiswa.

2.Marital status: Status pernikahan mahasiswa (single, married, dll).

3.Application mode: Cara atau jalur pendaftaran mahasiswa.

4.Application order: Urutan pilihan program studi yang diajukan saat mendaftar.

5.Course: Kode program studi yang diambil.

6.Daytime/evening attendance: Jenis kelas (day = reguler pagi, evening = kelas malam).

7.Previous qualification: Jenis kualifikasi pendidikan sebelumnya (misal SMA, diploma).

8.Previous qualification (grade): Nilai akhir dari pendidikan sebelumnya.

9.Nacionality: Kewarganegaraan mahasiswa.

10.Mother's qualification: Pendidikan terakhir ibu.

11.Father's qualification: Pendidikan terakhir ayah.

12.Mother's occupation: Pekerjaan ibu.

13.Father's occupation: Pekerjaan ayah.

14.Admission grade: Nilai ujian masuk universitas.

15.Displaced: Apakah mahasiswa tinggal jauh dari keluarga (1 = ya, 0 = tidak).

16.Educational special needs: Apakah mahasiswa memiliki kebutuhan khusus (1 = ya, 0 = tidak).

17.Debtor: Apakah mahasiswa memiliki tunggakan pembayaran (1 = ya, 0 = tidak).

18.Tuition fees up to date: Apakah pembayaran biaya kuliah sudah lunas.

19.Gender: Jenis kelamin mahasiswa.

20.Scholarship holder: Apakah mahasiswa adalah penerima beasiswa.

21.Age at enrollment: Umur mahasiswa saat mendaftar kuliah.

22.International: Status mahasiswa internasional (1 = ya, 0 = tidak).

23.Curricular units 1st sem (credited): Jumlah SKS semester 1 yang diakui dari tempat lain.

24.Curricular units 1st sem (enrolled): Jumlah SKS semester 1 yang diambil.

25.Curricular units 1st sem (evaluations): Jumlah evaluasi yang diikuti pada semester 1.

26.Curricular units 1st sem (approved): Jumlah SKS semester 1 yang lulus.

27.Curricular units 1st sem (grade): Rata-rata nilai mata kuliah semester 1.

28.Curricular units 1st sem (without evaluations): Jumlah SKS semester 1 tanpa evaluasi.

29.Curricular units 2nd sem (credited): Jumlah SKS semester 2 yang diakui.

30.Curricular units 2nd sem (enrolled): Jumlah SKS semester 2 yang diambil.

31.Curricular units 2nd sem (evaluations): Jumlah evaluasi yang diikuti semester 2.

32.Curricular units 2nd sem (approved): Jumlah SKS semester 2 yang lulus.

33.Curricular units 2nd sem (grade): Rata-rata nilai semester 2.

34.Curricular units 2nd sem (without evaluations): Jumlah SKS semester 2 tanpa evaluasi.

35.Unemployment rate: Tingkat pengangguran nasional pada saat pendaftaran.

36.Inflation rate: Tingkat inflasi nasional.

37.GDP: Pertumbuhan Produk Domestik Bruto (PDB) nasional.

38.Target: Label hasil akhir mahasiswa (Graduate, Dropout, Enrolled).


 **2️. Lakukan agregasi antara tabel features dengan tabel labels.**

In [18]:
# Menggabungkan menjadi satu
df = pd.concat([feature_data, label_data], axis=1)

# Membersihkan nama kolom dari karakter/spasi aneh dan menghapus kolom yang tidak relevan
df = df.rename(columns={
    'Unnamed: 0': 'ID',
    'Daytime/evening attendance\t': 'Daytime/evening attendance',
    'Nacionality': 'Nationality'
})
if 'Student Id' in df.columns:
    df = df.drop('Student Id', axis=1)

print("--- Data Setelah Digabung dan Dibersihkan ---")
display(df.head())

--- Data Setelah Digabung dan Dibersihkan ---


,ID,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,single,1.0,3.0,9773.0,day,1.0,NaN,Portuguese,1.0,...,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02,Graduate
1,1,single,1.0,1.0,NaN,day,1.0,136.0,Portuguese,19.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79,Graduate
2,2,single,42.0,1.0,9500.0,day,1.0,120.0,Portuguese,1.0,...,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70,Graduate
3,3,single,1.0,1.0,NaN,day,1.0,141.0,Portuguese,1.0,...,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02,Graduate
4,4,single,44.0,1.0,NaN,day,39.0,120.0,Spanish,4.0,...,1.0,6.0,NaN,3.0,12.000000,0.0,12.4,0.5,1.79,Dropout


**3️. Tampilkan tipe data dari setiap kolom yang terdapat dalam dataset.**

In [19]:
# Menampilkan informasi ringkas dari dataframe, termasuk tipe data dan jumlah data non-null
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ID                                              3096 non-null   int64  
 1   Marital status                                  3012 non-null   object 
 2   Application mode                                2924 non-null   float64
 3   Application order                               2988 non-null   float64
 4   Course                                          2944 non-null   float64
 5   Daytime/evening attendance                      3016 non-null   object 
 6   Previous qualification                          2973 non-null   float64
 7   Previous qualification (grade)                  3011 non-null   float64
 8   Nationality                                     2911 non-null   object 
 9   Mother's qualification                   

**4️. Tampilkan statistik deskriptif dari dataset dan berikan analisis singkat mengenai hasilnya.**

In [20]:
# Menampilkan statistik deskriptif untuk semua kolom (numerik dan objek)
display(df.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,3096.0,NaN,NaN,NaN,1547.5,893.882543,0.0,773.75,1547.5,2321.25,3095.0
Marital status,3012,6,single,2674,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Application mode,2924.0,NaN,NaN,NaN,18.785568,17.532674,1.0,1.0,17.0,39.0,53.0
Application order,2988.0,NaN,NaN,NaN,1.73494,1.327527,1.0,1.0,1.0,2.0,9.0
Course,2944.0,NaN,NaN,NaN,8817.929348,2148.407432,33.0,9085.0,9238.0,9556.0,9991.0
Daytime/evening attendance,3016,2,day,2701,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Previous qualification,2973.0,NaN,NaN,NaN,4.685503,10.439709,1.0,1.0,1.0,1.0,43.0
Previous qualification (grade),3011.0,NaN,NaN,NaN,132.647293,13.322662,95.0,124.0,133.1,140.0,190.0
Nationality,2911,19,Portuguese,2836,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mother's qualification,2951.0,NaN,NaN,NaN,19.47001,15.636952,1.0,2.0,19.0,37.0,44.0


Berdasarkan hasil df.describe(include='all').T, dapat diketahui hal-hal berikut:

Kolom numerik (seperti feature_1, feature_2, dst.) menampilkan ukuran statistik utama:

mean menunjukkan nilai rata-rata dari tiap variabel.

std (standard deviation) menunjukkan seberapa besar variasi atau penyebaran data dari rata-ratanya.

min, 25%, 50%, 75%, dan max menggambarkan sebaran data dari nilai terkecil hingga terbesar.

Kolom kategorikal (jika ada, misalnya gender, status, dsb.) menampilkan:

unique → jumlah kategori unik.

top → kategori yang paling sering muncul.

freq → frekuensi kemunculan kategori tersebut.

Dari hasil statistik, dapat dilihat bahwa:

Beberapa kolom numerik memiliki rata-rata yang berbeda cukup jauh dari median, yang mengindikasikan adanya kemungkinan data tidak terdistribusi normal (skewed).

Jika terdapat nilai minimum atau maksimum yang jauh dari kuartil atas/bawah, hal ini bisa menandakan adanya outlier.

Tidak semua kolom memiliki jumlah data yang sama, yang berarti mungkin ada missing value


**5️. Slicing Data**
**Tampilkan:**

**20 baris awal dari dataset**
**20 baris terakhir**
**Baris dari index 80 - 100**

In [21]:
# Menampilkan 20 baris pertama dari dataset
print("--- 20 Baris Awal ---")
display(df.head(20))

# Menampilkan 20 baris terakhir dari dataset
print("\n--- 20 Baris Terakhir ---")
display(df.tail(20))

# Menampilkan baris dari index 80 hingga 100
print("\n--- Baris dari Index 80 sampai 100 ---")
display(df.iloc[80:101])

--- 20 Baris Awal ---


,ID,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,single,1.0,3.0,9773.0,day,1.0,NaN,Portuguese,1.0,...,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02,Graduate
1,1,single,1.0,1.0,NaN,day,1.0,136.0,Portuguese,19.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79,Graduate
2,2,single,42.0,1.0,9500.0,day,1.0,120.0,Portuguese,1.0,...,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70,Graduate
3,3,single,1.0,1.0,NaN,day,1.0,141.0,Portuguese,1.0,...,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02,Graduate
4,4,single,44.0,1.0,NaN,day,39.0,120.0,Spanish,4.0,...,1.0,6.0,NaN,3.0,12.000000,0.0,12.4,0.5,1.79,Dropout
5,5,married,39.0,1.0,9853.0,day,1.0,NaN,Portuguese,NaN,...,0.0,7.0,10.0,3.0,13.000000,0.0,11.1,0.6,2.02,Dropout
6,6,single,39.0,1.0,9085.0,day,1.0,133.1,NaN,3.0,...,0.0,6.0,14.0,4.0,12.000000,0.0,11.1,0.6,2.02,Dropout
7,7,single,1.0,4.0,9670.0,day,1.0,130.0,Portuguese,1.0,...,0.0,6.0,7.0,6.0,13.428571,0.0,8.9,1.4,3.51,Enrolled
8,8,single,1.0,1.0,9773.0,day,1.0,135.0,Portuguese,1.0,...,0.0,6.0,7.0,6.0,12.166667,0.0,11.1,0.6,2.02,Graduate
9,9,single,39.0,1.0,9670.0,day,1.0,120.0,Portuguese,11.0,...,0.0,5.0,8.0,4.0,11.250000,0.0,7.6,2.6,0.32,Dropout



--- 20 Baris Terakhir ---


,ID,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
3076,3076,single,17.0,NaN,9254.0,day,1.0,122.0,Portuguese,1.0,...,0.0,6.0,11.0,3.0,11.333333,0.0,7.6,2.6,0.32,Enrolled
3077,3077,single,1.0,2.0,9500.0,day,1.0,140.0,Portuguese,NaN,...,0.0,8.0,8.0,8.0,13.881250,0.0,9.4,-0.8,-3.12,Graduate
3078,3078,single,1.0,1.0,9085.0,day,1.0,131.0,Portuguese,19.0,...,0.0,5.0,5.0,NaN,12.800000,0.0,10.8,1.4,1.74,Graduate
3079,3079,single,17.0,1.0,9238.0,day,1.0,120.0,Portuguese,19.0,...,0.0,6.0,13.0,2.0,10.666667,0.0,16.2,0.3,-0.92,Dropout
3080,3080,divorced,39.0,1.0,9003.0,day,12.0,133.1,Portuguese,37.0,...,10.0,13.0,14.0,13.0,14.230769,NaN,15.5,2.8,-4.06,Graduate
3081,3081,single,39.0,1.0,171.0,day,1.0,147.0,Portuguese,37.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,16.2,0.3,-0.92,Dropout
3082,3082,single,17.0,5.0,9147.0,day,1.0,131.0,Portuguese,37.0,...,0.0,5.0,11.0,5.0,11.400000,4.0,16.2,NaN,-0.92,Graduate
3083,3083,single,1.0,1.0,9500.0,day,1.0,140.0,Portuguese,19.0,...,0.0,8.0,11.0,8.0,14.881818,0.0,16.2,0.3,-0.92,Graduate
3084,3084,single,1.0,2.0,9773.0,day,1.0,130.0,Portuguese,1.0,...,0.0,6.0,6.0,6.0,12.333333,0.0,12.4,0.5,1.79,Graduate
3085,3085,single,17.0,1.0,9238.0,day,NaN,139.0,Portuguese,19.0,...,0.0,6.0,7.0,6.0,14.000000,0.0,15.5,2.8,-4.06,Graduate



--- Baris dari Index 80 sampai 100 ---


,ID,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
80,80,single,17.0,1.0,9670.0,day,1.0,155.0,Portuguese,1.0,...,NaN,6.0,10.0,4.0,13.000000,0.0,12.7,3.7,-1.70,Enrolled
81,81,single,39.0,1.0,9003.0,day,1.0,NaN,Portuguese,38.0,...,0.0,NaN,8.0,0.0,0.000000,0.0,15.5,2.8,-4.06,Dropout
82,82,single,1.0,1.0,171.0,day,1.0,159.0,NaN,3.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,NaN,1.79,Dropout
83,83,single,1.0,1.0,9147.0,day,1.0,150.0,Ukrainian,37.0,...,0.0,5.0,7.0,5.0,13.000000,0.0,9.4,-0.8,-3.12,Graduate
84,84,single,NaN,1.0,9991.0,evening,1.0,140.0,Portuguese,37.0,...,0.0,5.0,5.0,NaN,0.000000,0.0,15.5,2.8,-4.06,Dropout
85,85,single,18.0,1.0,9500.0,day,1.0,144.0,Portuguese,37.0,...,NaN,8.0,8.0,7.0,13.542857,0.0,12.7,3.7,-1.70,Graduate
86,86,single,1.0,1.0,9773.0,day,1.0,117.0,Portuguese,1.0,...,0.0,6.0,11.0,3.0,13.000000,0.0,15.5,2.8,-4.06,Graduate
87,87,single,39.0,2.0,9556.0,day,19.0,133.1,Portuguese,37.0,...,0.0,8.0,9.0,8.0,11.962500,0.0,13.9,-0.3,0.79,Graduate
88,88,single,1.0,2.0,9500.0,day,1.0,145.0,Portuguese,37.0,...,0.0,7.0,7.0,6.0,12.950000,0.0,7.6,2.6,0.32,Graduate
89,89,single,43.0,1.0,NaN,day,1.0,121.0,Portuguese,1.0,...,0.0,6.0,8.0,4.0,11.250000,0.0,9.4,NaN,-3.12,Enrolled


**6️. Tangani nilai yang hilang (NaN) menggunakan KNNImputer**

In [22]:
# --- TAMPILKAN DATA SEBELUM IMPUTASI ---
print("--- SEBELUM IMPUTASI ---")
print("Jumlah nilai kosong di setiap kolom:")
print(df.isnull().sum())
print("\nContoh 5 baris data yang memiliki nilai kosong (NaN):")
display(df[df.isnull().any(axis=1)].head())


# Membuat salinan data untuk diimputasi
df_imputed = df.copy()

# Memisahkan kolom numerik dan kategorikal
numeric_cols = df_imputed.select_dtypes(include=np.number).columns
categorical_cols = df_imputed.select_dtypes(include=['object']).columns

# Tahap 1: Imputasi Kolom Numerik dengan KNNImputer
print("\nMemulai proses imputasi numerik dengan KNNImputer...")
knn_imputer = KNNImputer(n_neighbors=5)
df_imputed[numeric_cols] = knn_imputer.fit_transform(df_imputed[numeric_cols])
print("Imputasi numerik selesai.")

# Tahap 2: Imputasi Kolom Kategorikal dengan Modus
print("Memulai imputasi kategorikal dengan Modus...")
from sklearn.impute import SimpleImputer
imputer_mode = SimpleImputer(strategy='most_frequent')
df_imputed[categorical_cols] = imputer_mode.fit_transform(df_imputed[categorical_cols])
print("Imputasi kategorikal selesai.")


# --- TAMPILKAN DATA SESUDAH IMPUTASI ---
print("\n--- SESUDAH IMPUTASI ---")
print("Jumlah nilai kosong di setiap kolom setelah proses:")
print(df_imputed.isnull().sum())
print("\nContoh 5 baris data setelah nilai kosong terisi:")
display(df_imputed.head())

--- SEBELUM IMPUTASI ---
Jumlah nilai kosong di setiap kolom:
ID                                                  0
Marital status                                     84
Application mode                                  172
Application order                                 108
Course                                            152
Daytime/evening attendance                         80
Previous qualification                            123
Previous qualification (grade)                     85
Nationality                                       185
Mother's qualification                            145
Father's qualification                             96
Mother's occupation                                62
Father's occupation                               144
Admission grade                                   141
Displaced                                         122
Educational special needs                         119
Debtor                                            116
Tuition fees up to d

,ID,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,single,1.0,3.0,9773.0,day,1.0,NaN,Portuguese,1.0,...,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02,Graduate
1,1,single,1.0,1.0,NaN,day,1.0,136.0,Portuguese,19.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79,Graduate
2,2,single,42.0,1.0,9500.0,day,1.0,120.0,Portuguese,1.0,...,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70,Graduate
3,3,single,1.0,1.0,NaN,day,1.0,141.0,Portuguese,1.0,...,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02,Graduate
4,4,single,44.0,1.0,NaN,day,39.0,120.0,Spanish,4.0,...,1.0,6.0,NaN,3.0,12.000000,0.0,12.4,0.5,1.79,Dropout



Memulai proses imputasi numerik dengan KNNImputer...
Imputasi numerik selesai.
Memulai imputasi kategorikal dengan Modus...
Imputasi kategorikal selesai.

--- SESUDAH IMPUTASI ---
Jumlah nilai kosong di setiap kolom setelah proses:
ID                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Previous qualification (grade)                    0
Nationality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                              

,ID,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,single,1.0,3.0,9773.0,day,1.0,136.0,Portuguese,1.0,...,0.0,6.0,6.0,6.0,15.333333,0.0,11.1,0.6,2.02,Graduate
1,1.0,single,1.0,1.0,5780.8,day,1.0,136.0,Portuguese,19.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,12.4,0.5,1.79,Graduate
2,2.0,single,42.0,1.0,9500.0,day,1.0,120.0,Portuguese,1.0,...,4.0,8.0,8.0,7.0,13.285714,0.0,12.7,3.7,-1.70,Graduate
3,3.0,single,1.0,1.0,9677.2,day,1.0,141.0,Portuguese,1.0,...,0.0,6.0,7.0,6.0,13.142857,0.0,11.1,0.6,2.02,Graduate
4,4.0,single,44.0,1.0,9194.4,day,39.0,120.0,Spanish,4.0,...,1.0,6.0,7.8,3.0,12.000000,0.0,12.4,0.5,1.79,Dropout


 Lakukan encoding terhadap kolom bertipe object menjadi numerik menggunakan LabelEncoder

In [23]:
# Menggunakan LabelEncoder untuk kolom target ('Target')
target_encoder = LabelEncoder()
df_imputed['Target'] = target_encoder.fit_transform(df_imputed['Target'])
print("--- Encoding untuk kolom 'Target' selesai ---")

# Menggunakan One-Hot Encoding (get_dummies) untuk sisa fitur kategorikal
object_features = df_imputed.select_dtypes(include=['object']).columns
df_final = pd.get_dummies(df_imputed, columns=object_features, drop_first=True)
print("--- Encoding untuk kolom fitur selesai ---")

# Menampilkan hasil transformasi setelah encoding
print("\nHasil transformasi setelah encoding:")
display(df_final.head())

--- Encoding untuk kolom 'Target' selesai ---
--- Encoding untuk kolom fitur selesai ---

Hasil transformasi setelah encoding:


,ID,Application mode,Application order,Course,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Nationality_Italian,Nationality_Lithuanian,Nationality_Moldova (Republic of),Nationality_Mozambican,Nationality_Portuguese,Nationality_Romanian,Nationality_Russian,Nationality_Santomean,Nationality_Spanish,Nationality_Ukrainian
0,0.0,1.0,3.0,9773.0,1.0,136.0,1.0,1.0,4.0,7.0,...,False,False,False,False,True,False,False,False,False,False
1,1.0,1.0,1.0,5780.8,1.0,136.0,19.0,19.0,4.0,8.0,...,False,False,False,False,True,False,False,False,False,False
2,2.0,42.0,1.0,9500.0,1.0,120.0,1.0,2.0,9.0,3.0,...,False,False,False,False,True,False,False,False,False,False
3,3.0,1.0,1.0,9677.2,1.0,141.0,1.0,19.0,4.0,4.0,...,False,False,False,False,True,False,False,False,False,False
4,4.0,44.0,1.0,9194.4,39.0,120.0,4.0,4.0,3.0,6.0,...,False,False,False,False,False,False,False,False,True,False


**8️. Tampilkan kembali dataset yang telah melalui seluruh proses preprocessing (cleaning, imputasi, dan encoding).**

In [24]:
print("--- Dataset Final Setelah Seluruh Proses Preprocessing ---")
display(df_final)

# Pemeriksaan terakhir pada struktur data final
print("\n--- Informasi Dataset Final ---")
df_final.info()

--- Dataset Final Setelah Seluruh Proses Preprocessing ---


,ID,Application mode,Application order,Course,Previous qualification,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,...,Nationality_Italian,Nationality_Lithuanian,Nationality_Moldova (Republic of),Nationality_Mozambican,Nationality_Portuguese,Nationality_Romanian,Nationality_Russian,Nationality_Santomean,Nationality_Spanish,Nationality_Ukrainian
0,0.0,1.0,3.0,9773.0,1.0,136.0,1.0,1.0,4.0,7.0,...,False,False,False,False,True,False,False,False,False,False
1,1.0,1.0,1.0,5780.8,1.0,136.0,19.0,19.0,4.0,8.0,...,False,False,False,False,True,False,False,False,False,False
2,2.0,42.0,1.0,9500.0,1.0,120.0,1.0,2.0,9.0,3.0,...,False,False,False,False,True,False,False,False,False,False
3,3.0,1.0,1.0,9677.2,1.0,141.0,1.0,19.0,4.0,4.0,...,False,False,False,False,True,False,False,False,False,False
4,4.0,44.0,1.0,9194.4,39.0,120.0,4.0,4.0,3.0,6.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091,3091.0,44.0,1.0,9085.0,1.0,140.0,4.0,4.0,2.0,6.0,...,False,False,False,False,True,False,False,False,False,False
3092,3092.0,1.0,5.0,9238.0,1.0,126.0,38.0,1.0,9.0,5.0,...,False,False,False,False,True,False,False,False,False,False
3093,3093.0,1.0,2.0,9500.0,1.0,141.0,37.0,37.0,9.0,7.0,...,False,False,False,False,True,False,False,False,False,False
3094,3094.0,39.0,1.0,9500.0,4.0,150.0,3.0,38.0,3.0,7.0,...,False,False,False,False,True,False,False,False,False,False



--- Informasi Dataset Final ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 59 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ID                                              3096 non-null   float64
 1   Application mode                                3096 non-null   float64
 2   Application order                               3096 non-null   float64
 3   Course                                          3096 non-null   float64
 4   Previous qualification                          3096 non-null   float64
 5   Previous qualification (grade)                  3096 non-null   float64
 6   Mother's qualification                          3096 non-null   float64
 7   Father's qualification                          3096 non-null   float64
 8   Mother's occupation                             3096 non-null   float64
 9   Father's